In [1]:
# preamble for nice notebooks
from IPython.core.interactiveshell import InteractiveShell; InteractiveShell.ast_node_interactivity = "all"  # multiple cell outputs

# Ecological Footprint, Happiness and Economic Growth

- data from https://happyplanetindex.org and https://footprintnetwork.org
- [The Happy Planet Index (HPI)](https://invidious.snopyta.org/watch?v=M1o3FS0awtk) strives a balance between ecological footprint and human happiness. 
- GDP is not a good measure for happiness and disregards non-human beings.

The Ecological Footprint should be taken with care, as it can lead to mutual __blaming__ between people that should instead _work together_. While shining a spotlight on overall environmental destruction, the focus on individuals instead of policies and systems can lead to __inability to act__.

The publicly downloadable global data from the footprint network is almost ten years old. In a previous study I found flaws in the datasets ontology. On a closer look, the data in iteself implies that global population growth is the main factor for the increase in ecological footprint. This is especially true for non-north-western countries, especially in Asia. This insight can foster racism while diverting from actual leverage points: how to transform our economic system to degrow.

When using the Happy Planet Index, it is important to keep in mind that the index is not a measure for happiness, but for the balance between ecological footprint and human happiness. The index is not a measure for the happiness of non-human beings. Happiness is hard to quantify, but __GDP is no measure for happiness at all and sets incentives for exploitation__. 

The dataset is only available as an excel file, where no sheet contains the data in a completely machine readable format! Therefore we must do some work by our selves.

## Data Aquisition and Preparation

In [2]:
!wget https://happyplanetindex.org/wp-content/themes/hpi/public/downloads/happy-planet-index-2006-2020-public-data-set.xlsx -O ../data/happy-planet-index.xlsx

import pandas as pl

data_ = pl.read_excel('../data/happy-planet-index.xlsx', dtype={'Country':str}, sheet_name='All data')
data_.head()
data_.dtypes

--2023-03-30 22:21:46--  https://happyplanetindex.org/wp-content/themes/hpi/public/downloads/happy-planet-index-2006-2020-public-data-set.xlsx
SSL_INIT
CA-Zertifikat »/etc/ssl/certs/ca-certificates.crt« wurde geladen
Auflösen des Hostnamens happyplanetindex.org (happyplanetindex.org)… 195.15.218.246
Verbindungsaufbau zu happyplanetindex.org (happyplanetindex.org)|195.15.218.246|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 1028610 (1005K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Wird in »../data/happy-planet-index.xlsx« gespeichert.

../data/happy-plane 100%[===================>]   1005K   269KB/s    in 3,7s    

2023-03-30 22:21:50 (269 KB/s) - »../data/happy-planet-index.xlsx« gespeichert [1028610/1028610]



,HPI rank,Country,ISO,Continent,Unnamed: 4,Population (thousands),Year,LifeExp (years),Ladder of life (0-10),Footprint (g ha),HPI,Biocapacity for year (g ha),GDP per capita ($)
0,NaN,Afghanistan,AFG,6.0,2020AFG,38928.341,2020.0,64.894261,NaN,NaN,NaN,1.55,..
1,15.0,Albania,ALB,7.0,2020ALB,2877.800,2020.0,78.082782,5.364910,1.789761,54.336054,1.55,..
2,84.0,United Arab Emirates,ARE,4.0,2020ARE,9890.400,2020.0,77.788521,6.458392,7.878637,35.626946,1.55,..
3,41.0,Argentina,ARG,1.0,2020ARG,45195.777,2020.0,75.202650,5.900567,3.047550,48.439379,1.55,..
4,NaN,Armenia,ARM,7.0,2020ARM,2963.234,2020.0,73.988411,NaN,NaN,NaN,1.55,..


HPI rank                       float64
Country                         object
ISO                             object
Continent                      float64
Unnamed: 4                      object
Population (thousands)         float64
Year                           float64
LifeExp (years)                float64
Ladder of life (0-10)          float64
Footprint (g ha)               float64
HPI                            float64
Biocapacity for year (g ha)    float64
GDP per capita ($)              object
dtype: object

In [3]:
data = data_.copy()

### Continent names from numbers 

# List is extracted from the excel file
continents = """\
Latin America
N America & Oceania
Western Europe
Middle East & N. Africa
Africa
South Asia
Eastern Europe & Central Asia
East Asia\
""".split('\n')

#> replace the value in the 'Continent' column by the value of the coresponding index in 'continents'
#> convert the 'Continent' value to int before to use it as an index 
#> convert float NaN to "None" 
data['Continent'] = data['Continent'].apply(lambda x: continents[int(x-1)] if not pl.isna(x) else "None")

#> and drop every invalid continent
data = data[data['Continent'].apply(lambda x: x in continents)]



### Floatify GDP per capita (was string)

def toFloat(s):
    try:    return float(s)
    except: return float('nan')

data['GDP per capita ($)'] = data['GDP per capita ($)'].apply(toFloat)


data.head()

,HPI rank,Country,ISO,Continent,Unnamed: 4,Population (thousands),Year,LifeExp (years),Ladder of life (0-10),Footprint (g ha),HPI,Biocapacity for year (g ha),GDP per capita ($)
0,NaN,Afghanistan,AFG,South Asia,2020AFG,38928.341,2020.0,64.894261,NaN,NaN,NaN,1.55,NaN
1,15.0,Albania,ALB,Eastern Europe & Central Asia,2020ALB,2877.800,2020.0,78.082782,5.364910,1.789761,54.336054,1.55,NaN
2,84.0,United Arab Emirates,ARE,Middle East & N. Africa,2020ARE,9890.400,2020.0,77.788521,6.458392,7.878637,35.626946,1.55,NaN
3,41.0,Argentina,ARG,Latin America,2020ARG,45195.777,2020.0,75.202650,5.900567,3.047550,48.439379,1.55,NaN
4,NaN,Armenia,ARM,Eastern Europe & Central Asia,2020ARM,2963.234,2020.0,73.988411,NaN,NaN,NaN,1.55,NaN


## Visualizing Happiness, Economic Grwoth and Ecological Footprint

In [4]:
import altair as alt
alt.renderers.enable('mimetype')  # enable static rendering

def worms_chart(x,y, note, title):
    return alt.Chart(data, title=title).mark_trail().encode(
        x=x, y=y, detail='Country',
        order='Year', color='Continent',
        tooltip=['Country', 'Year'],
        size=alt.Size( "Year", legend=None,
            scale=alt.Scale(domain=[2006,2020], range=[0.1, 5]))
    ).properties(width=800, height=400,
        title=alt.TitleParams(
            title,
            subtitle=f"{y.split(' ')[0]}  per  {x.split(' ')[0]}  by country (2006 to 2020 - thick is more recent).  {note}",
        ))

RendererRegistry.enable('mimetype')

In [5]:
worms_chart('GDP per capita ($)', 'Footprint (g ha)', "Red line is maximum sustainable footprint.",
    "Economic growth impacts the environment, but to different extents.") +\
alt.Chart(pl.DataFrame({'Footprint (g ha)':[1.8]})).mark_rule(color='red').encode(y='Footprint (g ha)')

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Countries like Mongolia and Turkmenistan (upper left) show that econmic growth can be more or less harmfull to the environment - it depends on the way of growth, which is determined by national policies and global market constrains. Qatar (upper right) shows that it is possible to 'degrow' while still increasing the environmental impact! Singapor, Luxembourg (right) and other northest European countrie showup with growth while reducing footprint. This is a sign of hope, but should be taken with care, as these countries historically contributed the most and now profit from externalization effects. But for most economies, especially in South Asia, economic growth is proportional to environmental destruction. Besides the most African countries all economies by far surpass the maximum sustainable footprint per person.
    
    Data from happyplanetindex.org. Red line number approximate from footprintnetwork.org (state of 2014!).

In [6]:
worms_chart('GDP per capita ($)', 'HPI', "", "Sustainable Happniness is unrelated to Economic Growth")

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [7]:
worms_chart('Footprint (g ha)', 'HPI', "", "Decreasing environmenal impact increases planet happiness")

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
